# 輸送最適化問題の解決

In [1]:
#!pip3 install pulp
#!pip3 install ortoolpy

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

df_tc = pd.read_csv("trans_cost.csv", index_col="工場")
df_demand = pd.read_csv("demand.csv")
df_supply = pd.read_csv("supply.csv")

In [3]:
df_tc.head()

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


In [4]:
df_demand.head()

,F1,F2,F3,F4
0,28,29,31,25


In [5]:
df_supply.head()

,W1,W2,W3
0,35,41,42


In [6]:
#初期設定
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))
print(nw)
print(nf)
print(pr)

3
4
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]


In [7]:
# 数理モデル作成 最小化するモデル
m1 = model_min()

v1 = {(i,j):LpVariable("v%d_%d"%(i,j),lowBound=0) for i, j in pr}
#print(v1)
# 目的関数の定義 輸送ルートコストの和
m1 += lpSum(df_tc.iloc[i][j]*v1[i,j] for i,j in pr)
# 制約条件
for i in range(nw):
    m1 += lpSum(v1[i,j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()

df_tr_sol = df_tc.copy()
total_cost = 0
for k,x in v1.items():
    i,j = k[0],k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j]*value(x)

print(df_tr_sol)
print("総輸送コスト:"+str(total_cost))

    F1  F2  F3  F4
工場                
W1  28   7   0   0
W2   0   0  31   5
W3   0  22   0  20
総輸送コスト:1296.0
